In [1]:
import checklist
import spacy
import itertools
import checklist.editor
import checklist.text_generation
import numpy as np
import random
import json
from checklist.model_api import test_model
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from checklist.pred_wrapper import PredictorWrapper

In [2]:
editor = checklist.editor.Editor()
nlp = spacy.load('zh_core_web_sm')
editor.tg

Building prefix dict from the default dictionary ...
08/23/2020 23:07:09 - DEBUG - jieba -   Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
08/23/2020 23:07:09 - DEBUG - jieba -   Loading model from cache /tmp/jieba.cache
Loading model cost 0.874 seconds.
08/23/2020 23:07:10 - DEBUG - jieba -   Loading model cost 0.874 seconds.
Prefix dict has been built successfully.
08/23/2020 23:07:10 - DEBUG - jieba -   Prefix dict has been built successfully.
08/23/2020 23:07:16 - INFO - transformers.tokenization_utils_base -   Model name '/work/QA_task/roberta-1.1/BERTCN/bertcn-pytorch-r1.1/checklist/checklist-master/checklist/data/model' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-who

In [6]:
import json
examples, labels, texts = [], [], []
label2id = json.load(open("/work/QA_task/roberta-1.1/BERTCN/bertcn-pytorch-r1.1/data/intent/new_label2id_680.json"))
for line in open("/work/QA_task/roberta-1.1/BERTCN/bertcn-pytorch-r1.1/data/intent/dev_680_train.txt").readlines()[:10]:
    line = line.strip('\n')
    lines = line.split('|')
    texts.append(line)
    examples.append(lines[0])
    labels.append(label2id[lines[1]])
labels = np.array(labels).astype(int)

In [7]:
parsed_examples = list(nlp.pipe(examples))
spacy_map = dict([(x, y) for x, y in zip(examples, parsed_examples)])
suite = TestSuite()
parsed_qs = [(spacy_map[q]) for q in examples]
parsed_qs[:2]

[对硬盘稳定性要求高, 西湖里面一般有啥吃的空格贵吗]

## Vocabulary

In [10]:
template = "我是要{v:mask}张伟。他是工程师"
verbs = editor.suggest(templates=template, )[:10]
print(', '.join(verbs))

找, 说, 黑, 问, 骂, 叫, 学, 请, 夸, 嫁


In [19]:
male = editor.data['names']['male']
data = editor.template('我是要{verb}{male},他是工程师',
                verb=verbs,
                male=male,
                remove_duplicates=True, 
                nsamples=100)
data[:3]
test = MFT(**t, labels=563, name='动词测试', capability='Vocabulary', 
          description = '填充不同动词，名字填充为男性，测试分类是否依然准确')
suite.add(test)

In [ ]:
female = editor.data['names']['female']
data = editor.template('我是要找{female},她是{post}',
                female=female,
                remove_duplicates=True, 
                nsamples=100)
data[:3]
test = MFT(**t, labels=563, name='职位测试', capability='Vocabulary', 
          description = '填充不同职位，名字填充为女性，测试分类是否依然准确')
suite.add(test)

In [ ]:
data = editor.template('我不是{company}的员工!', remove_duplicates=True, nsamples=50)
test = MFT(**t, labels=361, name='公司测试', capability='Vocabulary', 
          description = '填充不同公司，测试分类是否依然准确')
suite.add(test)

In [ ]:
mod = ['真的', '确实', '绝对', '毫无疑问', '确确实实']
data = editor.template('我{mod}不是{company}的员工!', mod=mod, remove_duplicates=True, nsamples=50)
test = MFT(**t, labels=361, name='修饰语测试', capability='Vocabulary', 
          description = '增加不同的修饰语，测试分类是否依然准确')
suite.add(test)

In [ ]:
adj = editor.suggest('我好{a:mask}啊，去休息室怎么走？')[:20]
data = editor.template('我好{adj}啊，去休息室怎么走？', adj=adj, remove_duplicates=True, nsamples=50)
test = MFT(**t, labels=495, name='形容词测试', capability='Vocabulary', 
          description = '使用不同的形容词，测试分类是否依然准确')
suite.add(test)

In [ ]:
followup = ['会伤身吗？', '会伤肾吗?','伤身体吗？', '可以一直吃吗?']
data = editor.template('蛋白粉吃多了会怎么样?{followup}', followup=followup, remove_duplicates=True, nsamples=20)
test = MFT(**t, labels=523, name='增加追问', capability='Vocabulary', 
          description = '增加一段追问，测试分类是否依然准确')
suite.add(test)

In [2]:
### Taxonomy

In [1]:
syn = []
x = editor.suggest('什么空气净化器比较{mask}?')
x += editor.suggest('什么空气净化器不太{mask}?')
for a in set(x):
    e = editor.synonyms('什么空气净化器%s?' % a, a)
    if e:
        syn.append([a] + e)
print(',\n'.join([str(tuple(x)) for x in tmp]))

NameError: name 'editor' is not defined

In [ ]:
ops = []
for a in set(x):
    e = editor.antonyms('什么空气净化器%s?' % a, a)
    if e:
        ops.append([a] + e)
print(',\n'.join([str(tuple(x)) for x in tmp]))

In [ ]:
temp = [y for x in syn for y in x] + [y for x in ops for y in x]
data = editor.template('什么空气净化器{syn}?', syn=temp, remove_duplicates=True, nsamples=50)
test = MFT(**t, labels=655, name='测试同义词和反义词', capability='Taxonomy', 
          description = '填充同义词或反义词，测试分类是否依然准确')
suite.add(test)

In [ ]:
t = Perturb.perturb(list(all_questions), apply_and_pair(replace_pairs(synonyms)), nsamples=1000, keep_original=False)
test = INV(t.data, threshold=0.1, name=name, description=desc, capability='Taxonomy')